# 逻辑斯蒂回归

## 使用MNIST数据集中的0，1数据完成logistic regression 任务

### 导入相关包

In [1]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn

In [2]:
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 定义数据预处理方式
transform = transforms.ToTensor()

利用框架自带数据集读取方式读取 MNIST数据集

In [3]:
# download 第一次可设置为True
trainset = tv.datasets.MNIST(
    root='./data/',
    train=True,
    download=True,
    transform=transform)

从训练集中取出0、1标签的数据构成此次任务的训练集(train_input,train_label) 

In [4]:
train_input, train_label = [], []

for cin, label in trainset:
    if label in [0, 1]:
        train_input.append(cin)
        train_label.append(label)

train_input = torch.stack(train_input)
train_label = torch.tensor(train_label)

In [5]:
# 定义测试数据集
testset = tv.datasets.MNIST(
    root='./data/',
    train=False,
    download=True,
    transform=transform)

从测试集中取出0、1标签的数据构成此次任务的测试集(test_input,test_lable)

In [6]:
test_input, test_label = [], []

for cin, label in testset:
    if label in [0, 1]:
        test_input.append(cin)
        test_label.append(label)
        
test_input = torch.stack(test_input)
test_label = torch.tensor(test_label)

 创建自己的数据集，需要补充完成

In [7]:
class TensorDataset(torch.utils.data.Dataset):
    """
    创建自己的dataset类 继承torch.utils.data.Dataset。
    Dataset wrapping data and target tensors.
    Each sample will be retrieved by indexing both tensors along the first
    dimension.
    Arguments:
        data_tensor (Tensor): contains sample data.
        target_tensor (Tensor): contains sample targets (labels).
    """

    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor

    # 这个方法是必须要有的，用于按照索引读取每个元素的具体内容
    def __getitem__(self, index):
        # #return很关键，return回哪些内容，那么我们在训练时循环读取每个batch时，
        # 就能获得哪些内容
        return self.data_tensor[index], self.target_tensor[index]

    ##这个函数也必须要写，它返回的是数据集的长度
    def __len__(self):
        return self.data_tensor.size(0)

定义训练批处理数据

In [8]:
# batch_size 根据自己计算资源设计，shuffle设置为True，查询设置的原因。
trainloader = torch.utils.data.DataLoader(
    TensorDataset(train_input, train_label),
    batch_size=64,
    shuffle=True,
    )
# #shuffle参数用于控制数据的洗牌方式，当shuffle设置为True时，每次读取数据都会
# 对数据进行随机洗牌，保证训练过程中数据的顺序是随机的，从而避免模型对某些特定
# 顺序的数据过拟合。因此，设置shuffle=True可以提高模型的泛化能力

# 测试集
testloader = torch.utils.data.DataLoader(
    TensorDataset(test_input, test_label),
    batch_size=32,
    shuffle=True,
    )

利用pytorch框架的nn.Module 模块完成logistic regression 模型的设计

In [9]:
class logistic_regression(nn.Module):
    '''
        逻辑回归模型
    '''
    def __init__(self):
        super(logistic_regression, self).__init__()
        ### 添加logistic regression层
        self.linear = nn.Linear(28*28, 1)

    def forward(self, x):
        ### 计算图（1）数据维度拉长
        #        （2）logistic regression层传播
        #        （3）return输出（注意维度）
        
        # 数据维度拉长
        x = x.view(-1, self.linear.in_features)
        # 逻辑斯蒂层传播
        output = self.linear(x)
        # return输出
        return torch.sigmoid(output)

In [10]:
# 实例化模型
log_reg = logistic_regression().to(device)

设置损失函数及优化器。提示：使用binary cross entropy 损失函数，优化器可设置SGD、Adam等

In [11]:
# 使用binary cross entropy损失函数
criterion = nn.BCELoss()

# 设置优化器
optimizer = torch.optim.SGD(log_reg.parameters(), lr=0.0001)

训练

In [12]:
for epoch in range(5):
    sum_loss = 0.0
    # 数据读取
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # 梯度清零
        optimizer.zero_grad()
        
        # 模型输出
        outputs = log_reg(inputs)
        
        # 定义loss
        loss = criterion(outputs, labels.float().view(-1, 1))
        
        # loss反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        
        sum_loss += loss.item()

    print('第{0}轮 loss: {1}'.format(epoch+1, sum_loss / 100))

#  输出测试集准确率: 没有对测试集进行DataLoader，如果自己电脑配置不够，
#尝试建立testloader 然后按照批量计算
    with torch.no_grad():
        test_outputs = log_reg(test_input.cuda())

        test_predicted = torch.squeeze(test_outputs>=0.5).int()
        
        accuracy = (test_predicted.cpu() == test_label.int()).sum().float() / test_label.size()[0]
        print('识别准确率为：{}%'.format(accuracy*100))

第1轮 loss: 1.3040272545814515
识别准确率为：81.8439712524414%
第2轮 loss: 1.200778603553772
识别准确率为：92.90780639648438%
第3轮 loss: 1.1122282963991166
识别准确率为：97.54137420654297%
第4轮 loss: 1.0353444460034371
识别准确率为：99.38534545898438%
第5轮 loss: 0.9679196089506149
识别准确率为：99.57447052001953%
